### **PHISHING VECTOR GENERATOR** 🐟

In [5]:
import numpy as np
import requests
import re
from urllib.parse import urlparse
from os import path
from tld import get_tld
from bs4 import BeautifulSoup
from phishing_utils import *
from user_browsing import user_browsing
from difflib import SequenceMatcher
from html import unescape

In [6]:
class PHISH_FVG:

    def __init__(self, url, tfidf):

        self.url = url
        parsed = urlparse(url)
        self.base = parsed.netloc
        self.path = parsed.path

        self.fv = np.array([-1 for i in range(19)])

        self.user = user_browsing()

        response_content = get_bin_source_code(
            self.url,
            self.user.get_simple_user_header_agent(),
            self.user.proxies)

        content = response_content.decode("utf-8", errors='ignore')
        self.html = unescape(content)
        self.soup = BeautifulSoup(response_content)

        self.hyperlinks = find_hyperlinks_tags(self.soup)
        self.tfidf = tfidf

    def set_feature_vector(self):

        self.set_f1()
        self.set_f2()
        self.set_f3()
        self.set_f4()
        self.set_f5()
        self.set_f6()
        self.set_f7()
        self.set_f8()
        self.set_f9()
        self.set_f10_f11()
        self.set_f12()
        self.set_f13()
        self.set_f14()
        self.set_f15()
        self.set_f16()
        self.set_f17()
        self.set_f18()
        self.set_f19()

    def set_f1(self):
        """
        Sets F1.
        F1 = 1, if dots in url >= 4
        F1 = 0, otherwise
        """

        if self.url.count('.') >= 4:
            self.fv[0] = 1

        else:
            self.fv[0] = 0

    def set_f2(self):
        """
        Sets F2.
        F2 = 1, if URL contains '@' or '-' symbols
        F2 = 0, otherwise
        """

        if '@' in self.url or '-' in self.url:
            self.fv[1] = 1

        else:
            self.fv[1] = 0

    def set_f3(self):
        """
        Sets F3.
        F3 = 1, if URL length >= 74
        F3 = 0, otherwise
        """

        if len(self.url) >= 74:
            self.fv[2] = 1

        else:
            self.fv[2] = 0

    def set_f4(self):
        """
        Sets F4.
        F4 = 1, if URL contains any suspicious word
        F4 = 0, otherwise
        """

        splitted_url = get_splitted_url(self.url)
        suspicious_words = get_suspicious_keywords()

        for word in splitted_url:
            leet_translation = translate_leet_to_letters(
                word)  # Decisión propia

            if bool(suspicious_words & leet_translation):
                self.fv[3] = 1
                return

        self.fv[3] = 0

    def set_f5(self):
        """
        Sets F5.
        F5 = 1, if tlds in URL > 1
        F5 = 0, otherwise
        """

        tld = get_tld(self.url, fix_protocol=True)
        base_without_tld = self.base[:-len(tld) - 1]
        rest = base_without_tld + self.path

        splitted_url = get_splitted_url_keep_dots(rest)
        tlds = get_tlds_set()
        tlds = set(['.' + tld for tld in tlds])

        if len(splitted_url & tlds) >= 1:
            self.fv[4] = 1

        else:
            self.fv[4] = 0

    def set_f6(self):
        """
        Sets F6.
        F6 = 1, if http count in URL > 1
        F6 = 0, otherwise
        """

        if len(re.findall('http', self.url)) > 1:
            self.fv[5] = 1

        else:
            self.fv[5] = 0

    def set_f7(self):
        """
        Sets F7.
        F7 = 1, if brand in incorrect position.
        F7 = 0, otherwise
        """

        targets = get_phishing_targets_set()

        lower_url = self.url.lower()
        parsed = urlparse(lower_url)
        path = parsed.path

        for target in targets:
            if target in path:  # or target in sub_domains
                self.fv[6] = 1
                return

        # Extra checking - leet translation wherever except base
        # domain and tld but not exact word

        base = parsed.netloc
        tld = get_tld(lower_url, fix_protocol=True)
        without_tld = base[:-len(tld) - 1]

        if without_tld.count('.') > 0:
            sub_domains = without_tld[:without_tld.rindex('.')]
        else:
            sub_domains = without_tld

        for word in get_splitted_url(path + sub_domains):
            leet_translation = translate_leet_to_letters(
                word)  # Decisión propia

            # If the original one does not have numbers it is removed
            if not re.search(r'\d', word):
                leet_translation -= {word}

            for fake in leet_translation:
                for target in targets:

                    if SequenceMatcher(None, fake, target).ratio() >= 0.8:
                        self.fv[6] = 1
                        return

        self.fv[6] = 0

    def set_f8(self):
        """
        Sets F8.
        F8 = 1, if data URI present in website.
        F8 = 0, otherwise
        """

        matches = find_data_URIs(self.html)

        if len(matches) > 0:
            self.fv[7] = 1

        else:
            self.fv[7] = 0

    def set_f9(self):
        """
        Sets F9.
        F9 = 1, if action field is blank or javascript:void(0)
        F9 = 1, if action field is <name>.php
        F9 = 1, if action field contains foreign base domain
        F9 = 0, otherwise
        """

        forms_found = re.findall("<form[^>]+>", self.html)

        if len(forms_found) > 0:

            for i in range(len(forms_found)):
                form_found = forms_found[i]
                action_content = re.findall(
                    '(?:action=\")([^"]*)(?:\")', form_found)

                if len(action_content) > 0:

                    if is_empty(action_content[0]):
                        self.fv[8] = 1
                        return

                    elif is_simple_php_file(action_content[0]):
                        self.fv[8] = 1
                        return

                    elif is_foreign(self.url, action_content[0]):
                        self.fv[8] = 1
                        return

        self.fv[8] = 0

    def set_f10_f11(self):
        """
        Sets F10 and F11.

        F10 = number of hyperlinks in source code.

        F11 = 1, if no hyperlinks found in source.
        F11 = 0, otherwise
        """

        n_hyperlinks_found = len(self.hyperlinks)
        self.fv[9] = n_hyperlinks_found

        if n_hyperlinks_found == 0:
            self.fv[10] = 1

        else:
            self.fv[10] = 0

    def set_f12(self):
        """
        Sets F12.

        ratio = |n_foreign_hyp| / |n_hyp|

        F12 = 1 if ratio > 0.5 and n_hyp > 0
        F12 = 0 otherwise
        """

        if len(self.hyperlinks) == 0:
            self.fv[11] = 0  # Opino que debería ser 1
            return

        n_foreigns = get_number_foreign_hyperlinks(self.url, self.hyperlinks)
        ratio = n_foreigns / len(self.hyperlinks)

        if ratio > 0.5:
            self.fv[11] = 1
        else:
            self.fv[11] = 0

    def set_f13(self):
        """
        Sets F13.

        ratio = |n_empty_hyp| / |n_hyp|

        F13 = 1 if ratio > 0.34 and n_hyp > 0
        F13 = 0 otherwise
        """

        if len(self.hyperlinks) == 0:
            self.fv[12] = 0  # Opino que debería ser 1
            return

        n_empty = get_number_empty_hyperlinks(self.hyperlinks)
        ratio = n_empty / len(self.hyperlinks)

        if ratio > 0.34:
            self.fv[12] = 1
        else:
            self.fv[12] = 0

    def set_f14(self):
        """
        Sets F14.

        ratio = |n_errors_hyp| / |n_hyp|

        F14 = 1 if ratio > 0.3 and n_hyp > 0
        F14 = 0 otherwise
        """

        if len(self.hyperlinks) == 0:
            self.fv[13] = 0  # Opino que debería ser 1
            return

        n_errors = get_number_errors(self.hyperlinks,
                                     self.user.get_simple_user_header_agent(),
                                     self.user.proxies)

        ratio = n_errors / len(self.hyperlinks)

        if ratio > 0.3:
            self.fv[13] = 1
        else:
            self.fv[13] = 0

    def set_f15(self):
        """
        Sets F15.

        ratio = |n_redirects| / |n_hyp|

        F15 = 1 if ratio > 0.3 and n_hyp > 0
        F15 = 0 otherwise
        """

        if len(self.hyperlinks) == 0:
            self.fv[14] = 0  # Opino que debería ser 1
            return

        n_redirects = get_number_redirects(self.hyperlinks,
                                           self.user.get_simple_user_header_agent(),
                                           self.user.proxies)
        ratio = n_redirects / len(self.hyperlinks)

        if ratio > 0.3:
            self.fv[14] = 1
        else:
            self.fv[14] = 0

    def set_f16(self):
        """
        Sets F16.

        F16 = 1, if CSS file is external and contains foreign domain name
        F16 = 0, otherwise

        #'assets/bootstrap/css/bootstrap.min.css' foreign?
        """

        external_csss = self.soup.findAll("link", rel="stylesheet")

        for css in external_csss:

            link = extract_url_href(css)

            if is_foreign(self.url, link):
                self.fv[15] = 1
                return

        self.fv[15] = 0

    def set_f17(self):
        """
        Sets F17.
        F17 = 0 if copyright keyword matches base domain
        F17 = 1, otherwise
        """

        copyright_clues = ['©', '&#169', '& copy', '&copy', 'copy', 'copyright',
                           'copyright', 'all right reserved', 'rights', 'right']  # '@',

        for clue in copyright_clues:

            regex = '(?:{})([^<.>"\']*)(?:[<.>"\'])'.format(clue)
            copy_contents = re.findall(regex, self.html)

            for copy_content in copy_contents:
                copy_content = remove_punctuation(copy_content).reshape(1)

                for content in copy_content[0].split():
                    # Avoid single letters or small strings
                    if len(content) > 2 and re.findall(content.replace(",", ""), self.base, re.IGNORECASE):
                        self.fv[16] = 0
                        return

        self.fv[16] = 1

    def set_f18(self):
        """
        Set F18.
        F18 = 1 if no keyword matches domain name
        F18 = 0 Otherwise
        """

        keywords = get_site_keywords(self.html, self.tfidf, 15)

        for keyword in keywords:

            if re.findall(keyword, self.base):
                self.fv[17] = 0
                return

        self.fv[17] = 1

    def set_f19(self):
        """
        Sets F19.
        F19 = 1, if foreign domain found in favicon link
        F19 = 0, otherwise
        """

        icons = self.soup.findAll("link", rel="icon")
        icons += self.soup.findAll("link", rel="shortcut icon")

        for icon in icons:

            link = extract_url_href(icon)

            if is_foreign(self.url, link):
                self.fv[18] = 1
                return

        self.fv[18] = 0


# **TESTS**

In [7]:
import unittest
import os
import sys

src_path = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(src_path)


class RealFV(unittest.TestCase):

    def setUp(self):
        self.ph_entity = PHISH_FVG('https://ubuvirtual.ubu.es/', None)

    def test_proxy_working(self):

        ip_one = requests.get('http://ipinfo.io/ip',
                              proxies=self.ph_entity.user.proxies).text
        ip_two = requests.get('http://ipinfo.io/ip').text
        self.assertTrue(ip_one != ip_two)


if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)


.
----------------------------------------------------------------------
Ran 1 test in 0.835s

OK


# **TF-IDF**

In [8]:
user = user_browsing()

urls = get_csv_data(get_data_path() + os.sep + 'alexa_filtered.csv')[:200]

corpus = get_tfidf_corpus(
    urls, user.get_simple_user_header_agent(), user.proxies)
tfidf = get_tfidf(corpus)

# **GENERATION OF VECTORS**

## **REALS**

In [9]:
# reales = get_alexa_sites()
# fvs_real = []

# for real in reales:

#     try:
#         ph_entity = PHISH_FVG(real, tfidf)
#         ph_entity.set_feature_vector()
#         fvs_real.append(np.append(ph_entity.fv, [0]))

#     except:
#         print(f"Error en {real}")

# output_file = get_fv_path() + os.path.sep + 'genuine_fv.csv'

# with open(output_file, mode='w') as f:

#     writer = csv.writer(f, delimiter=',', quotechar='"',
#                         quoting=csv.QUOTE_MINIMAL)
#     writer.writerow([f"f{i}" for i in range(1,20)] + ['tag'])

#     for url in fvs_real:
#         writer.writerow(url)

# f.close()

## **PHISHING**

In [10]:
phishings = get_phish_tank_urls_csv(10000000).union(get_open_fish_urls())

In [11]:
fvs_phishing = []

for phishing in phishings:

    try:
        ph_entity = PHISH_FVG(phishing, tfidf)
        ph_entity.set_feature_vector()
        fvs_phishing.append(np.append(ph_entity.fv, [1]))

        if len(fvs_phishing) == 1100:
            break

    except: #(ConnectionError, requests.exceptions.SSLError, requests.exceptions.ConnectTimeOut):
        print(phishing)

output_file = get_fv_path() + os.path.sep + 'phishing_fv.csv'

with open(output_file, mode='w') as f:

    writer = csv.writer(f, delimiter=',', quotechar='"',
                        quoting=csv.QUOTE_MINIMAL)
    writer.writerow([f"f{i}" for i in range(1,20)] + ['tag'])

    for url in fvs_phishing:
        writer.writerow(url)

f.close()


http://msghnkebks.duckdns.org
http://help-center604.crabdance.com/verify
http://help-center243.crabdance.com/verify


/home/patripata/anaconda3/lib/python3.9/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


https://ffzxdcsa.top/jp
https://ylegktmtly.duckdns.org/
http://help-center202.crabdance.com/verify
https://consenti-disconoscimento.com/
http://allrecipelibrary.com
https://formatclientibp.com/
https://thejcdesign.com/bper/
https://visualizando-limiteonline.com/luiza/?userID=13&uri=WcQctpzQ6GlufyJ6Xm4Gt77vAAU/09ZeGGIODh7iocA=
https://amazon-ivtsdab.dns-report.com/
http://help-center543.crabdance.com/verify
https://ctznsvr1fy.com/login
https://shoopets.com/
https://goofy-mirzakhani.213-232-235-24.plesk.page/oran/
https://voba-energiepraemie-2022.xyz/s/anmeldung.php
http://successful-mip.com
https://connection-orangebank-certicode.derlma.com/9745238/
https://dkb-info-kuden.istmein.de/auth/realms/login-dkb/
https://esiminsfr.com/verification/login.php
https://web9665.web07.bero-webspace.de/Tele2/
https://here.meship.top/
https://etc-mdsgfd.dsmtp.com/
https://amazon-uytrdfg.youdontcare.com/
http://abonnement-sfr-esim.com/
https://cof.felain.top/
http://dappregistration.online
https://panne

/home/patripata/anaconda3/lib/python3.9/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


http://pancakeswap.com.cm
https://paiements-amendes.fr/
https://esim-demande.fr/
https://amendesdossiergouv.com/
https://xn--mtamask-2xa.net/
http://help-center287.crabdance.com/verify
http://info-conto-online.162-0-237-145.cprapid.com/checkclient.php?&amp;sessionid=9d30164cdeb61a4caaff2d044a3fd599
http://www.jensenhome.net.dream.website/wp-admin/opttioons.html
https://id.fatturazione-syspec.com/
https://www.casasubshow.com/335c44e287f60d03c9eb7a150/checkout/order/?PaymentMethod=billet&amp;orderId=03-1027706386
https://cuatroesquinasarte.com/
http://help-center435.crabdance.com/verify
http://help-center494.crabdance.com/verify
http://ripristinoanomalia-app.com/
https://post.emirateesdelivvery.net
https://s.micscseuocl-jp.icu/
https://connection-orangebank-certicode.derlma.com/3895247/
http://frontyardcompanyio.live
http://help-center533.crabdance.com/verify
https://contrattorestrittivoutentesconosciuto.com/
https://accessofindomestic.nomobeauty.com/
https://qsmx.eu.org/made/
http://fan